Для начала обучим нейросеть для распознавания лиц на датасете CelebA-500.

Для этого будем дообучать нейросеть ResNet50, обученную на датасете ImageNet.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import numpy as np
import math
import PIL
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm, tqdm_notebook

from torchvision.models import resnet18, resnet50

In [2]:
# Работа была проведена на локальной машине, поэтому подключение Google Disk не требуется
# from google.colab import drive

# drive.mount('/content/drive')

In [3]:
image_size = 224

# Определим аугментацию для тренировки и теста модели
transform_train = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3),
    transforms.RandomGrayscale(p=0.3),
    # transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3)),
    transforms.Normalize(
        (0.485, 0.456, 0.406),
        (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize([image_size, image_size]),
    transforms.ToTensor(),
    transforms.Normalize(
        (0.485, 0.456, 0.406),
        (0.229, 0.224, 0.225)),
])

path_to_anno = './celebA_train_500/celebA_anno.txt'
path_to_split = './celebA_train_500/celebA_train_split.txt'
path_to_imgs = './celebA_train_500/celebA_imgs/'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
from collections import defaultdict

# file with query part annotations: which image belongs to which class
# format:
#     image_name_1.jpg 2678
#     image_name_2.jpg 2679

f = open(path_to_anno, 'r')
img_lines = f.readlines()
f.close()
img_lines = [x.strip('\n').split(' ') for x in img_lines]

f = open(path_to_split, 'r')
img_train_split = f.readlines()
f.close()
img_train_split = [x.strip('\n').split(' ') for x in img_train_split]

# plain list of image names from query. Needed to compute embeddings for query
img_names = [x[0] for x in img_lines]
img_labels = [int(x[1]) for x in img_lines]
img_split = [int(x[1]) for x in img_train_split]

for idx in range(len(img_lines)):
    img_lines[idx].append(img_split[idx])


In [5]:
# Определим тренировачные, валидационные и тестовые данные из файла celebA_anno.txt

x_train, x_val, x_test, y_train, y_val, y_test = [], [], [], [], [], []

np.random.shuffle(img_lines)
for line in img_lines:
    if line[2] == 0:
        x_train.append(line[0])
        y_train.append(int(line[1]))
    elif line[2] == 1:
        x_val.append(line[0])
        y_val.append(int(line[1]))
    elif line[2] == 2:
        x_test.append(line[0])
        y_test.append(int(line[1]))

In [6]:
print(len(x_train), len(x_val), len(x_test))

8544 1878 1589


In [7]:
class FacesDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, labels, mode, transform=transform_train):
        super().__init__()
        # список файлов для загрузки
        self.files = files

        self.len_ = len(self.files)

        self.labels = torch.LongTensor(labels)

        if mode == 'train':
            self.transform = transform_train
        else:
            self.transform = transform_test

    def __len__(self):
        return self.len_

    def __getitem__(self, index):
        # load image
        cur_path = path_to_imgs + self.files[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transform(cur_img)

        return cur_img, self.labels[index]

    def show_img(self, index):
        # load image
        cur_path = path_to_imgs + self.files[index]
        cur_img = PIL.Image.open(cur_path).convert('RGB')
        cur_img = self.transform(cur_img)
        cur_img.show()

In [8]:
# Будем размораживать слои на 13 эпохе
def adjust_freezing(model, epoch):
    layers_to_unfreeze = [model.layer3, model.layer4, model.avgpool]
    if epoch == 13:
        for layer in layers_to_unfreeze:
            for param in layer.parameters():
                param.requires_grad = True
        summary(model, input_size=(3, 224, 224))

def fit_epoch(model, train_loader, criterion, optimizer, scheduler):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)

    scheduler.step()
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

def train(train_dataset, val_dataset, model, epochs, batch_size, optimizer, scheduler):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        criterion = nn.CrossEntropyLoss()

        for epoch in range(epochs):
            adjust_freezing(model, epoch)
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, optimizer, scheduler)

            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))

            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [9]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []

        for inputs in test_loader:
            inputs = inputs.to(device)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)

    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [10]:
def test_model(model, test_dataset):
    model.eval()
    running_corrects = 0
    processed_size = 0
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    test_acc = running_corrects.cpu().numpy() / processed_size
    return test_acc

In [11]:
# Дообучим модель resnet50
n_classes = len(np.unique(img_labels))

model = resnet50(pretrained=True)
fc_in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(fc_in_features, n_classes)
)

model.to(device)

C:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
# Заморозим первые 9 слоев сети, т.к. они выделяют низкоуровневые признаки, обучение с нуля которых
# может ухудшить качество наших предсказаний
counter = 0

for child in model.children():
    print(child)
    counter += 1
    if counter < 9:
        for param in child.parameters():
            param.requires_grad = False
print(counter)

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, t

In [13]:
#  Посмотрим на слои модели и удостоверимся, что первые 9 из них заморожены
model.to(device)
summary(model, input_size=(3, 224, 224))
print(1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [14]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, 1, 0.1, 10)

train_dataset = FacesDataset(x_train, y_train, mode='train')
val_dataset = FacesDataset(x_val, y_val, mode='val')
test_dataset = FacesDataset(x_test, y_test, mode='test')

In [15]:
epochs = 40
batch_size = 128

history = train(train_dataset, val_dataset, model, epochs, batch_size, optimizer, scheduler)

epoch:   2%|█▉                                                                          | 1/40 [01:06<43:19, 66.66s/it]


Epoch 001 train_loss: 6.4214     val_loss 5.7288 train_acc 0.0085 val_acc 0.0229


epoch:   5%|███▊                                                                        | 2/40 [02:15<42:53, 67.71s/it]


Epoch 002 train_loss: 5.4574     val_loss 5.2632 train_acc 0.0545 val_acc 0.0836


epoch:   8%|█████▋                                                                      | 3/40 [03:24<42:06, 68.28s/it]

...


Epoch 038 train_loss: 0.0132     val_loss 1.2934 train_acc 0.9999 val_acc 0.7226


epoch:  98%|█████████████████████████████████████████████████████████████████████████▏ | 39/40 [48:21<01:17, 77.05s/it]


Epoch 039 train_loss: 0.0119     val_loss 1.3028 train_acc 1.0000 val_acc 0.7231


epoch: 100%|███████████████████████████████████████████████████████████████████████████| 40/40 [49:39<00:00, 74.48s/it]


Epoch 040 train_loss: 0.0113     val_loss 1.2925 train_acc 0.9996 val_acc 0.7284


In [16]:
# Сохраняем модель при необходимости
# torch.save(model.state_dict(), 'model_weights.pth')

# Очищаем память от посчитанных градиентов
torch.cuda.empty_cache()

In [15]:
# Загружаем модель при необходимости
# model.load_state_dict(torch.load('model_weights.pth', weights_only=True))

<All keys matched successfully>

In [17]:
# Тестируем модель
test_acc = test_model(model, test_dataset)
print(f'Test Accuracy: {test_acc:.2f}')

Test Accuracy: 0.71


In [18]:
# Если вы работаете с данными, которые даны по ссылке,
# то эта ячейка поможет их загрузить
from collections import defaultdict
import os 
# file with query part annotations: which image belongs to which class
# format:
#     image_name_1.jpg 2678
#     image_name_2.jpg 2679
f = open('./celebA_ir/celebA_anno_query.csv', 'r')
query_lines = f.readlines()[1:]
f.close()
query_lines = [x.strip().split(',') for x in query_lines]
# plain list of image names from query. Neede to compute embeddings for query
query_img_names = [x[0] for x in query_lines]

# dictionary with info of which images from query belong to which class
# format:
#     {class: [image_1, image_2, ...]}
query_dict = defaultdict(list)
for img_name, img_class in query_lines:
  query_dict[img_class].append(img_name)

# list of distractor images
distractors_img_names = os.listdir('./celebA_ir/celebA_distractors')

In [19]:
def compute_embeddings(model, images_list):
    '''
    compute embeddings from the trained model for list of images.
    params:
    model: trained nn model that takes images and outputs embeddings
    images_list: list of images paths to compute embeddings for
    output:
    list: list of model embeddings. Each embedding corresponds to images
          names from images_list
    '''
    embeddings = []
    model.eval()
    with torch.no_grad():
        for idx in range(len(images_list)):
            img_name = images_list[idx]
            if images_list == query_img_names:
                image = PIL.Image.open('celebA_ir/celebA_query/'+img_name).convert('RGB')
            elif images_list == distractors_img_names:
                image = PIL.Image.open('celebA_ir/celebA_distractors/'+img_name).convert('RGB')
            else:
                raise Exception('Invalid list').with_traceback(traceback_obj)

            image = transform_test(image)
            image = image.to(device)
            output = model(image.unsqueeze(0))
            embeddings.append(output.detach().cpu())
    
    return torch.cat(embeddings).numpy()

In [20]:
query_embeddings = compute_embeddings(model, query_img_names)
distractors_embeddings = compute_embeddings(model, distractors_img_names)

In [21]:
def compute_cosine_query_pos(query_dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between positive pairs from query (stage 1)
    params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
    output:
    list of floats: similarities between embeddings corresponding
                    to the same people from query list
    '''
    
    img_to_emb = dict(zip(query_img_names, query_embeddings))
    result = []
    for idx, values in query_dict.items():
        for i in range(len(values)-1):
            for j in range(i+1, len(values)):
                cos_sim = cosine_similarity(np.array(img_to_emb[values[i]]).reshape(1,-1), np.array(img_to_emb[values[j]]).reshape(1,-1))
                result.append(cos_sim[0][0])
    return np.array(result)

def compute_cosine_query_neg(query_dict, query_img_names, query_embeddings):
    '''
    compute cosine similarities between negative pairs from query (stage 2)
    params:
    query_dict: dict {class: [image_name_1, image_name_2, ...]}. Key: class in
                the dataset. Value: images corresponding to that class
    query_img_names: list of images names
    query_embeddings: list of embeddings corresponding to query_img_names
    output:
    list of floats: similarities between embeddings corresponding
                    to different people from query list
    '''
    img_to_emb = dict(zip(query_img_names, query_embeddings))
    result = []
    keys = list(query_dict.keys())
    for i in range(len(keys)-1):
        values = query_dict[keys[i]]
        temp = []
        for j in range(i+1, len(keys)):
            for img in query_dict[keys[j]]:
                temp.append(img)
        for j in range(len(values)):
            for k in range(len(temp)):
                cos_sim = cosine_similarity(np.array(img_to_emb[values[j]]).reshape(1,-1), np.array(img_to_emb[temp[k]]).reshape(1,-1))
                result.append(cos_sim[0][0])
            
    return np.array(result)

def compute_cosine_query_distractors(query_embeddings, distractors_embeddings):
    '''
    compute cosine similarities between negative pairs from query and distractors
    (stage 3)
    params:
    query_embeddings: list of embeddings corresponding to query_img_names
    distractors_embeddings: list of embeddings corresponding to distractors_img_names
    output:
    list of floats: similarities between pairs of people (q, d), where q is
                    embedding corresponding to photo from query, d —
                    embedding corresponding to photo from distractors
    '''
    return np.array(cosine_similarity(query_embeddings, distractors_embeddings)).flatten()

In [22]:
cosine_query_pos = compute_cosine_query_pos(query_dict, query_img_names,
                                            query_embeddings)
cosine_query_neg = compute_cosine_query_neg(query_dict, query_img_names,
                                            query_embeddings)
cosine_query_distractors = compute_cosine_query_distractors(query_embeddings,
                                                            distractors_embeddings)


Ячейка ниже проверяет, что код работает верно:

In [23]:
test_query_dict = {
    2876: ['1.jpg', '2.jpg', '3.jpg'],
    5674: ['5.jpg'],
    864:  ['9.jpg', '10.jpg'],
}
test_query_img_names = ['1.jpg', '2.jpg', '3.jpg', '5.jpg', '9.jpg', '10.jpg']
test_query_embeddings = [
                    [1.56, 6.45,  -7.68],
                    [-1.1 , 6.11,  -3.0],
                    [-0.06,-0.98,-1.29],
                    [8.56, 1.45,  1.11],
                    [0.7,  1.1,   -7.56],
                    [0.05, 0.9,   -2.56],
]

test_distractors_img_names = ['11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg']

test_distractors_embeddings = [
                    [0.12, -3.23, -5.55],
                    [-1,   -0.01, 1.22],
                    [0.06, -0.23, 1.34],
                    [-6.6, 1.45,  -1.45],
                    [0.89,  1.98, 1.45],
]

test_cosine_query_pos = compute_cosine_query_pos(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_neg = compute_cosine_query_neg(test_query_dict, test_query_img_names,
                                            test_query_embeddings)
test_cosine_query_distractors = compute_cosine_query_distractors(test_query_embeddings,
                                                            test_distractors_embeddings)

In [24]:
true_cosine_query_pos = [0.8678237233650096, 0.21226104378511604,
                         -0.18355866977496182, 0.9787437979250561]
assert np.allclose(sorted(test_cosine_query_pos), sorted(true_cosine_query_pos)), \
      "A mistake in compute_cosine_query_pos function"

true_cosine_query_neg = [0.15963231223161822, 0.8507997093616965, 0.9272761484302097,
                         -0.0643994061127092, 0.5412660901220571, 0.701307100338029,
                         -0.2372575528216902, 0.6941032794522218, 0.549425446066643,
                         -0.011982733001947084, -0.0466679194884999]
assert np.allclose(sorted(test_cosine_query_neg), sorted(true_cosine_query_neg)), \
      "A mistake in compute_cosine_query_neg function"

true_cosine_query_distractors = [0.3371426578637511, -0.6866465610863652, -0.8456563512871669,
                                 0.14530087113136106, 0.11410510307646118, -0.07265097629002357,
                                 -0.24097699660707042,-0.5851992679925766, 0.4295494455718534,
                                 0.37604478596058194, 0.9909483738948858, -0.5881093317868022,
                                 -0.6829712976642919, 0.07546364489032083, -0.9130970963915521,
                                 -0.17463101988684684, -0.5229363015558941, 0.1399896725311533,
                                 -0.9258034013399499, 0.5295114163723346, 0.7811585442749943,
                                 -0.8208760031249596, -0.9905139680301821, 0.14969764653247228,
                                 -0.40749654525418444, 0.648660814944824, -0.7432584300096284,
                                 -0.9839696492435877, 0.2498741082804709, -0.2661183373780491]
assert np.allclose(sorted(test_cosine_query_distractors), sorted(true_cosine_query_distractors)), \
      "A mistake in compute_cosine_query_distractors function"

И, наконец, финальная функция, которая считает IR metric:

In [25]:
def compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors,
               fpr=0.1):
    '''
    compute identification rate using precomputer cosine similarities between pairs
    at given fpr
    params:
    cosine_query_pos: cosine similarities between positive pairs from query
    cosine_query_neg: cosine similarities between negative pairs from query
    cosine_query_distractors: cosine similarities between negative pairs
                              from query and distractors
    fpr: false positive rate at which to compute TPR
    output:
    float: threshold for given fpr
    float: TPR at given FPR
    '''
    N = int(fpr * (len(cosine_query_neg) + len(cosine_query_distractors)))
    false_pairs = np.concatenate([cosine_query_neg, cosine_query_distractors])
    false_pair = np.sort(false_pairs)[::-1]
    threshold = false_pair[N] 
    cosine_query_pos = np.array(cosine_query_pos)
    TPR = len(cosine_query_pos[cosine_query_pos >= threshold]) / len(cosine_query_pos) 
    return threshold, TPR

И ячейки для ее проверки:

In [26]:
test_thr = []
test_tpr = []
for fpr in [0.5, 0.3, 0.1]:
  x, y = compute_ir(test_cosine_query_pos, test_cosine_query_neg,
                    test_cosine_query_distractors, fpr=fpr)
  test_thr.append(x)
  test_tpr.append(y)

In [27]:
true_thr = [-0.011982733001947084, 0.3371426578637511, 0.701307100338029]
assert np.allclose(np.array(test_thr), np.array(true_thr)), "A mistake in computing threshold"

true_tpr = [0.75, 0.5, 0.5]
assert np.allclose(np.array(test_tpr), np.array(true_tpr)), "A mistake in computing tpr"

А в ячейке ниже вы можете посчитать TPR@FPR для датасета с лицами. Давайте, например, посчитаем для значений fpr = [0.5, 0.2, 0.1, 0.05].

In [28]:
fprs = [0.5, 0.2, 0.1, 0.05]

for fpr in fprs:
    x, y = compute_ir(cosine_query_pos, cosine_query_neg, cosine_query_distractors, 
               fpr)
    print(f'threshold for {fpr}: {y:.2f}')


threshold for 0.5: 0.86
threshold for 0.2: 0.59
threshold for 0.1: 0.42
threshold for 0.05: 0.29


In [29]:
# Очищаем память
torch.cuda.empty_cache()

In [30]:
class ArcMarginProduct(nn.Module):
    '''
    Implement of large margin arc distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
        cos(theta + m)
    '''
    def __init__(self, in_features, out_features, s=30.0, m=2, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        
    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

In [31]:
# Дообучим модель на основе модели ResNet50
model_afl = resnet50(pretrained=True)

# Заменим fc слой, а также добавим слой ArcMarginProduct
fc_in_features = model_afl.fc.in_features
model_afl.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(fc_in_features, n_classes)
)
model_afl.fc1 = ArcMarginProduct(n_classes, n_classes)

model_afl.to(device)

# Заморозим первые 9 слоев сети, т.к. они выделяют низкоуровневые признаки, обучение с нуля которых
# может ухудшить качество наших предсказаний
counter = 0

for child in model_afl.children():
    print(child)
    counter += 1
    if counter < 9:
        for param in child.parameters():
            param.requires_grad = False
print(counter)

C:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
ReLU(inplace=True)
MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, t

In [32]:
# Обучим модель с изменямым LR на первых 10 эпохах
optimizer = torch.optim.Adam(model_afl.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, 1, 0.1, 10)

epochs = 40
batch_size = 128

history = train(train_dataset, val_dataset, model_afl, epochs, batch_size, optimizer, scheduler)

epoch:   2%|█▉                                                                          | 1/40 [01:05<42:19, 65.12s/it]


Epoch 001 train_loss: 6.4148     val_loss 5.7432 train_acc 0.0095 val_acc 0.0261


epoch:   5%|███▊                                                                        | 2/40 [02:10<41:18, 65.21s/it]


Epoch 002 train_loss: 5.4475     val_loss 5.2844 train_acc 0.0520 val_acc 0.0724


epoch:   8%|█████▋                                                                      | 3/40 [03:16<40:24, 65.52s/it]

...


Epoch 038 train_loss: 0.0137     val_loss 1.2990 train_acc 1.0000 val_acc 0.7226


epoch:  98%|█████████████████████████████████████████████████████████████████████████▏ | 39/40 [47:49<01:17, 77.80s/it]


Epoch 039 train_loss: 0.0126     val_loss 1.2959 train_acc 0.9999 val_acc 0.7295


epoch: 100%|███████████████████████████████████████████████████████████████████████████| 40/40 [49:06<00:00, 73.67s/it]


Epoch 040 train_loss: 0.0124     val_loss 1.2861 train_acc 0.9999 val_acc 0.7258


In [33]:
# Тестируем модель
test_acc = test_model(model_afl, test_dataset)
print(f'Test Accuracy: {test_acc:.2f}')

Test Accuracy: 0.72


In [34]:
# Сохраняем модель при необходимости
# torch.save(model_afl.state_dict(), 'model_weights_arcfaceloss.pth')